In [ ]:
import cv2 as cv
import math

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
net = cv.dnn.readNetFromTensorflow("graph_opt.pb")

In [ ]:
inWidth = 368
inHeight = 368
thr = 0.2

In [ ]:
    BODY_PARTS = { "Nose": 0, "Neck": 1, "RShoulder": 2, "RElbow": 3, "RWrist": 4,
                   "LShoulder": 5, "LElbow": 6, "LWrist": 7, "RHip": 8, "RKnee": 9,
                   "RAnkle": 10, "LHip": 11, "LKnee": 12, "LAnkle": 13, "REye": 14,
                   "LEye": 15, "REar": 16, "LEar": 17, "Background": 18 }
    BODY_PART1 = { "Nose": 0 }

    POSE_PAIRS = [ ["Neck", "RShoulder"], ["Neck", "LShoulder"], ["RShoulder", "RElbow"],
                   ["RElbow", "RWrist"], ["LShoulder", "LElbow"], ["LElbow", "LWrist"],
                   ["Neck", "RHip"], ["RHip", "RKnee"], ["RKnee", "RAnkle"], ["Neck", "LHip"],
                   ["LHip", "LKnee"], ["LKnee", "LAnkle"], ["Neck", "Nose"], ["Nose", "REye"],
                   ["REye", "REar"], ["Nose", "LEye"], ["LEye", "LEar"] ]
    print(len(BODY_PARTS))
    print(len(BODY_PART1))

In [ ]:
BODY_PART2 = { "NE": 1 }
element = next(iter(BODY_PART2))
print(element)
print(len(element))

In [ ]:
img = cv.imread("lean_pose1.jpg")

In [ ]:
plt.imshow(img)

In [ ]:
plt.imshow(cv.cvtColor(img, cv.COLOR_BGR2RGB))

In [ ]:
def pose_estimation(frame):
    frameWidth = frame.shape[1]
    frameHeight = frame.shape[0]
    inp = cv.dnn.blobFromImage(frame, 1.0, (inWidth, inHeight),(127.5, 127.5, 127.5), swapRB=True, crop=False)
    net.setInput(inp)
    out = net.forward()
    out = out[:, :19, :, :]
    
    assert(len(BODY_PARTS) == out.shape[1])

    points = []
    for i in range(len(BODY_PARTS)):
        # Slice heatmap of corresponding body's part.
        heatMap = out[0, i, :, :]

        # Originally, we try to find all the local maximums. To simplify a sample
        # we just find a global one. However only a single pose at the same time
        # could be detected this way.
        _, conf, _, point = cv.minMaxLoc(heatMap)
        x = (frameWidth * point[0]) / out.shape[3]
        y = (frameHeight * point[1]) / out.shape[2]
        print("x=",x)
        print("y=",y)
        # Add a point if it's confidence is higher than threshold.
        points.append((int(x), int(y)) if conf > thr else None)
        
    for i in range(len(BODY_PART1)):
        heatMap = out[0, i, :, :]
        _, conf, _, point = cv.minMaxLoc(heatMap)
        x1 = (frameWidth * point[0]) / out.shape[3]
        y1 = (frameHeight * point[1]) / out.shape[2]
        print("       ")
        print("x1=",x1)
        print("y1=",y1)
    for i in range(len(element)):
        heatMap = out[0, i, :, :]
        _, conf, _, point = cv.minMaxLoc(heatMap)
        x2 = (frameWidth * point[0]) / out.shape[3]
        y2 = (frameHeight * point[1]) / out.shape[2]
        print("       ")
        print("x2=",x2)
        print("y2=",y2)
        print(" ")
    distance = ((((x2-x1)**2)+((y2-y1)**2))**0.5)
    print("the distance between them is ", distance)
    for pair in POSE_PAIRS:
        partFrom = pair[0]
        partTo = pair[1]
        assert(partFrom in BODY_PARTS)
        assert(partTo in BODY_PARTS)

        idFrom = BODY_PARTS[partFrom]
        idTo = BODY_PARTS[partTo]
        
        if points[idFrom] and points[idTo]:
            cv.line(frame, points[idFrom], points[idTo], (0, 0, 0), 3)
            cv.ellipse(frame, points[idFrom], (3, 3), 0, 0, 360, (255, 255, 255), cv.FILLED)
            cv.ellipse(frame, points[idTo], (3, 3), 0, 0, 360, (0, 0, 255), cv.FILLED)

    t, _ = net.getPerfProfile()
    freq = cv.getTickFrequency() / 1000
    cv.putText(frame, '%.2fms' % (t / freq), (10, 20), cv.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0))
    return frame
    return distance

In [ ]:
expected_image = pose_estimation(img)

In [ ]:
plt.imshow(cv.cvtColor(expected_image, cv.COLOR_BGR2RGB))

In [ ]:
lean = cv.VideoCapture('lean.gif')
lean.set(3,800)
lean.set(4,800)

if not lean.isOpened():
    lean = cv.VideoCapture(0)
if not lean.isOpened():
    raise IOError("can't open the video file")
    
while cv.waitKey(1) < 0:
    hasFrame,frame = lean.read()
    if not hasFrame:
        cv.WaitKey()
        break
    frameWidth = frame.shape[1]
    frameHeight = frame.shape[0]
    inp = cv.dnn.blobFromImage(frame, 1.0, (inWidth, inHeight),(127.5, 127.5, 127.5), swapRB=True, crop=False)
    net.setInput(inp)
    out = net.forward()
    out = out[:, :19, :, :]
    
    assert(len(BODY_PARTS) == out.shape[1])

    points = []
    for i in range(len(BODY_PARTS)):
        # Slice heatmap of corresponding body's part.
        heatMap = out[0, i, :, :]

        # Originally, we try to find all the local maximums. To simplify a sample
        # we just find a global one. However only a single pose at the same time
        # could be detected this way.
        _, conf, _, point = cv.minMaxLoc(heatMap)
        x = (frameWidth * point[0]) / out.shape[3]
        y = (frameHeight * point[1]) / out.shape[2]
        # Add a point if it's confidence is higher than threshold.
        points.append((int(x), int(y)) if conf > thr else None)
        
    for i in range(len(BODY_PART1)):
        heatMap = out[0, i, :, :]
        _, conf, _, point = cv.minMaxLoc(heatMap)
        x1 = (frameWidth * point[0]) / out.shape[3]
        y1 = (frameHeight * point[1]) / out.shape[2]
        print("       ")
        print("x1=",x1)
        print("y1=",y1)
        
    for i in range(len(element)):
        heatMap = out[0, i, :, :]
        _, conf, _, point = cv.minMaxLoc(heatMap)
        x2 = (frameWidth * point[0]) / out.shape[3]
        y2 = (frameHeight * point[1]) / out.shape[2]
        print("       ")
        print("x2=",x2)
        print("y2=",y2)
    distance = ((((x2-x1)**2)+((y2-y1)**2))**0.5)
    print("the distance between them is ", distance)
    
    for i in range(len('lean.gif')):
        if distance <71:
            text = "bend pose detected: {}".format(distance)
            cv.putText(frame, text, (10, frame.shape[0] - 25),cv.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0),2)
            print("detected")
        else:
            text1 = "bend pose not detected: {}".format(distance)
            cv.putText(frame, text1, (10, frame.shape[0] - 25),cv.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0),2)
            print("not detected")

    for pair in POSE_PAIRS:
        partFrom = pair[0]
        partTo = pair[1]
        assert(partFrom in BODY_PARTS)
        assert(partTo in BODY_PARTS)

        idFrom = BODY_PARTS[partFrom]
        idTo = BODY_PARTS[partTo]
        if points[idFrom] and points[idTo]:
            cv.line(frame, points[idFrom], points[idTo], (0, 255, 0), 3)
            cv.ellipse(frame, points[idFrom], (3, 3), 0, 0, 360, (0, 0, 255), cv.FILLED)
            cv.ellipse(frame, points[idTo], (3, 3), 0, 0, 360, (0, 0, 255), cv.FILLED)

    t, _ = net.getPerfProfile()
    freq = cv.getTickFrequency() / 1000
    cv.putText(frame, '%.2fms' % (t / freq), (10, 20), cv.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0))
    
    cv.imshow('video executed successfully ', frame)
  
    

In [ ]:
lean = cv.VideoCapture(1)
lean1 = cv.VideoCapture('video2.mp4')

lean.set(cv.CAP_PROP_FPS,10)
lean.set(3,800)
lean.set(4,800)

if not lean.isOpened():
    lean = cv.VideoCapture(0)
if not lean.isOpened():
    raise IOError("can't access and open webcam")
    
while cv.waitKey(1) < 0:
    hasFrame,frame = lean.read()
    if not hasFrame:
        cv.WaitKey()
        break
    frameWidth = frame.shape[1]
    frameHeight = frame.shape[0]
    inp = cv.dnn.blobFromImage(frame, 1.0, (inWidth, inHeight),(127.5, 127.5, 127.5), swapRB=True, crop=False)
    net.setInput(inp)
    out = net.forward()
    out = out[:, :19, :, :]
    
    assert(len(BODY_PARTS) == out.shape[1])

    points = []
    for i in range(len(BODY_PARTS)):
        # Slice heatmap of corresponding body's part.
        heatMap = out[0, i, :, :]

        # Originally, we try to find all the local maximums. To simplify a sample
        # we just find a global one. However only a single pose at the same time
        # could be detected this way.
        _, conf, _, point = cv.minMaxLoc(heatMap)
        x = (frameWidth * point[0]) / out.shape[3]
        y = (frameHeight * point[1]) / out.shape[2]

        # Add a point if it's confidence is higher than threshold.
        points.append((int(x), int(y)) if conf > thr else None)
                
    for i in range(len(BODY_PART1)):
        heatMap = out[0, i, :, :]
        _, conf, _, point = cv.minMaxLoc(heatMap)
        x1 = (frameWidth * point[0]) / out.shape[3]
        y1 = (frameHeight * point[1]) / out.shape[2]
        print("       ")
        print("x1=",x1)
        print("y1=",y1)
        
    for i in range(len(element)):
        heatMap = out[0, i, :, :]
        _, conf, _, point = cv.minMaxLoc(heatMap)
        x2 = (frameWidth * point[0]) / out.shape[3]
        y2 = (frameHeight * point[1]) / out.shape[2]
        print("       ")
        print("x2=",x2)
        print("y2=",y2)
    distance = ((((x2-x1)**2)+((y2-y1)**2))**0.5)
    print("the distance between them is ", distance)
    
    for i in range(len('video2.mp4')):
        if distance <100:
            text = "bend pose detected: {}".format(distance)
            cv.putText(frame, text, (10, frame.shape[0] - 25),cv.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0),2)
            print("detected")
        else:
            text1 = "bend pose not detected: {}".format(distance)
            cv.putText(frame, text1, (10, frame.shape[0] - 25),cv.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0),2)
            print("not detected")

    for pair in POSE_PAIRS:
        partFrom = pair[0]
        partTo = pair[1]
        assert(partFrom in BODY_PARTS)
        assert(partTo in BODY_PARTS)

        idFrom = BODY_PARTS[partFrom]
        idTo = BODY_PARTS[partTo]

        if points[idFrom] and points[idTo]:
            cv.line(frame, points[idFrom], points[idTo], (0, 255, 0), 3)
            cv.ellipse(frame, points[idFrom], (3, 3), 0, 0, 360, (0, 0, 255), cv.FILLED)
            cv.ellipse(frame, points[idTo], (3, 3), 0, 0, 360, (0, 0, 255), cv.FILLED)

    t, _ = net.getPerfProfile()
    freq = cv.getTickFrequency() / 1000
    cv.putText(frame, '%.2fms' % (t / freq), (10, 20), cv.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0))
    
    cv.imshow('LIVE WEBCAM ', frame)